In [1]:
import openmc 
import pandas as pd
import openmc.deplete as od
import math

## VOYGR model
The VOYGR modeled is simplified by using a PWR assembly model, specifically the [example model 
in OpenMC](https://docs.openmc.org/en/latest/pythonapi/generated/openmc.examples.pwr_assembly.html#openmc.examples.pwr_assembly).
The Fuel material is modified to be 4.09% weight fraction U-235, in a UO2 form. This modification 
matches the input fuel for the VOYGR used in the transition scenarios.

In [3]:
pwr_model = openmc.examples.pwr_assembly()

pwr_model.materials[0].nuclides.clear()
pwr_model.materials[0].add_nuclide("U235", 0.0404, 'ao')
pwr_model.materials[0].add_nuclide("U238", 0.9596, 'ao')
pwr_model.materials[0].add_nuclide("O16", 2, 'ao')
fuel = pwr_model.materials[0]

openmc.config['cross_sections'] = "/home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/cross_sections.xml"

In [5]:
voygr_cross_sections = od.MicroXS.from_model(pwr_model, 
                                       domain = fuel, 
                                       chain_file="./voygr/chain_casl_pwr.xml")

hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                #########

 Reading In113 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/In113.h5
 Reading In115 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/In115.h5
 Reading Sb121 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sb121.h5
 Reading Sb123 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sb123.h5
 Reading Sb125 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Sb125.h5
 Reading Te127_m1 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Te127_m1.h5
 Reading Te129_m1 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Te129_m1.h5
 Reading Te132 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Te132.h5
 Reading I127 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/I127.h5
 Reading I128 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/I128.h5
 Reading I129 from
 /home/abachmann@anl.go

 Reading Gd154 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Gd154.h5
 Reading Gd155 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Gd155.h5
 Reading Gd156 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Gd156.h5
 Reading Gd157 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Gd157.h5
 Reading Gd158 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Gd158.h5
 Reading Gd159 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Gd159.h5
 Reading Gd160 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Gd160.h5
 Reading Tb159 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Tb159.h5
 Reading Tb160 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Tb160.h5
 Reading Tb161 from
 /home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/neutron/Tb161.h5
 Reading Dy160 from
 /home/abachmann@anl.gov/openm

In [6]:
voygr_cross_sections.to_csv("voygr/micro_xs.csv")

In [7]:
voygr_cross_sections

,"(n,gamma)","(n,p)","(n,a)","(n,2n)","(n,3n)","(n,4n)",fission
nuclide,,,,,,,
B10,0.033834,0.003271,328.291549,0.000000,0.000000e+00,0.0,0.000000
B11,0.000473,0.000006,0.000063,0.000007,0.000000e+00,0.0,0.000000
O16,0.000046,0.000062,0.003478,0.000000,0.000000e+00,0.0,0.000000
Br81,1.468614,0.000087,0.000006,0.001682,0.000000e+00,0.0,0.000000
Kr82,6.227460,0.000054,0.000015,0.001259,0.000000e+00,0.0,0.000000
...,...,...,...,...,...,...,...
Cm242,3.754088,0.000000,0.000000,0.000534,2.615737e-07,0.0,1.331052
Cm243,14.051949,0.000000,0.000000,0.001443,1.882888e-06,0.0,85.430061
Cm244,17.816035,0.000000,0.000000,0.001191,9.476176e-06,0.0,0.939304


### Calculate Pu equivalence for MOX recipe

In [9]:
nu_235 = 2.4355
nu_238 = 2.819
nu_239 = 2.836

In [17]:
p_235 = nu_235*voygr_cross_sections.loc['U235']['fission'] - voygr_cross_sections.loc['U235']['(n,gamma)']
p_238 = nu_238*voygr_cross_sections.loc['U238']['fission'] - voygr_cross_sections.loc['U238']['(n,gamma)']
p_239 = nu_239*voygr_cross_sections.loc['Pu239']['fission'] - voygr_cross_sections.loc['Pu239']['(n,gamma)']

In [18]:
equivalence = (p_235-p_238)/(p_239-p_238)

In [21]:
Pu_enrich = 4.09/equivalence
Pu_enrich # weight fraction of the actinides

9.083466631774755

## Xe-100 model 

This data is based on the Serpent model of the Sanagmon200. Serpent generated the cross 
section data, which then was converted to the correct form for OpenMC

In [2]:
file = "./htgr-mr-full-core.inp_mdx0.m"
serpent_data = pd.read_csv(file, skiprows=1, skipfooter=2,delimiter=' ', usecols=[0,3,10], header=None)

/tmp/ipykernel_4146752/302349059.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  serpent_data = pd.read_csv(file, skiprows=1, skipfooter=2,delimiter=' ', usecols=[0,3,10], header=None)


In [3]:
openmc_data = pd.DataFrame(columns=['nuclide','(n,gamma)','(n,2n)','(n,p)','(n,a)','(n,3n)','(n,4n)','fission'])

In [4]:
for ii in range(0, 3318,6):
    zai = int(serpent_data.iloc[ii][0])
    Z= math.floor(zai/10000)
    A=math.floor((zai-Z*10000)/10)
    m = zai -(Z*10000 +A*10)
    nucname = openmc.data.gnds_name(Z,A,m)
    dictionary = {'nuclide':nucname, '(n,gamma)':serpent_data.iloc[ii][10],
                  '(n,2n)':serpent_data.iloc[ii+1][10],'(n,p)':serpent_data.iloc[ii+2][10],
                  '(n,a)':serpent_data.iloc[ii+3][10],'(n,3n)':serpent_data.iloc[ii+4][10],
                  '(n,4n)':0.0, 'fission':serpent_data.iloc[ii+5][10]}
    openmc_data.loc[len(openmc_data)] = dictionary
# append on last nuclide
openmc_data.loc[len(openmc_data)] = {'nuclide':'Es255', '(n,gamma)':0,
                  '(n,2n)':0,'(n,p)':0,
                  '(n,a)':serpent_data.iloc[ii+3][10],'(n,3n)':0,
                  '(n,4n)':0.0, 'fission':0}
openmc_data = openmc_data.set_index('nuclide')

In [5]:
openmc_data.to_csv("./xe-100/micro_xs.csv")

In [34]:
xe100_cross_sections = od.MicroXS.from_csv("./xe-100/micro_xs.csv")

### Calculate Pu equivalence for MOX

In [33]:
nu_235 = 2.4355
nu_238 = 2.819
nu_239 = 2.836

In [35]:
p_235 = nu_235*xe100_cross_sections.loc['U235']['fission'] - xe100_cross_sections.loc['U235']['(n,gamma)']
p_238 = nu_238*xe100_cross_sections.loc['U238']['fission'] - xe100_cross_sections.loc['U238']['(n,gamma)']
p_239 = nu_239*xe100_cross_sections.loc['Pu239']['fission'] - xe100_cross_sections.loc['Pu239']['(n,gamma)']

In [36]:
equivalence = (p_235-p_238)/(p_239-p_238)

In [37]:
Pu_enrich = 15.5/equivalence
Pu_enrich # weight fraction of the actinides

17.99848388689712